In [ ]:
import pystac_client
from oxeo.core.models.tile import load_tile_from_stac_as_dict, tile_from_id
from oxeo.core.models import data
URL = "https://earth-search.aws.element84.com/v0"
catalog = pystac_client.Client.open(URL)

In [ ]:
# It is possible to load a tile from a catalog and 
# return the selected slices and bands.  
# It uses tile.bbox_wgs84 to perform the search
# If chunkaligned is set to True then 
# if the tile aoi is bigger than one chunk it will bring another complete chunk
# It you find a rasterio/gdal error about PROJ. Run in you terminal:
# unset PROJ_LIB and relaunch your notebook
tile_id = "35_K_10000_56_758"
tile = tile_from_id(tile_id)
sample = load_tile_from_stac_as_dict(URL,["sentinel-s2-l2a-cogs"],
                                     tile, slice(0,6),
                                     ["B04", "B03", "B02"], 
                                     chunk_aligned=False, resolution=10)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.imshow(np.median(sample["image"],axis=0).transpose(1,2,0)/10000)
plt.show()

In [ ]:

# We can also create a search_params dict 
# and get a dask aoi that is, this time, chunkaligned with the original data source

search_params = {
                 "bbox":[-66.9235610961914, 
                         -39.76210275375138, 
                         -66.75533294677734, 
                         -39.64865819884837], #min_x, min_y, max_x, max_y
                 "collections":["sentinel-s2-l2a-cogs"],
                 "datetime":"2020-04-01/2020-05-01"
                 }
aoi = data.get_aoi_from_stac_catalog(catalog_url=URL, 
                                     search_params=search_params,
                                     chunk_aligned=True)
aoi


In [ ]:
# and perform selections on the aoi
aoi = aoi.sel(band=["B04", "B03", "B02"])
aoi = aoi.isel(time=[1,2,6])
aoi = aoi.median("time", keep_attrs=True)
aoi

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(aoi.values.transpose(1,2,0)/10000)
plt.show()